In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import helpers

import geopandas as gpd

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
pstname =  'mango'
fold =  'mango_v3'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
# m_d = f_d = "C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_flash"
m_d = f_d = r"C:\GSP\sv\model\update_2024\scripts\pestaws_mango"
pst = pyemu.Pst(os.path.join(f_d,f"{pstname}.pst"))

# pst = pyemu.Pst(os.path.join(f_d,f"flash.pst"))

In [ ]:
os.listdir(m_d)

In [ ]:
rei = pd.read_csv(os.path.join(m_d, f"{pstname}.mean.rei"), sep = '\s+', skiprows=[0,1,2,])

In [ ]:
sfr = rei.query("Group.str.contains('kenwood_log')")
sfr

In [ ]:

if rei.query("Group.str.contains('aguacal_log')").shape==rei.query("Group.str.contains('aguacal_log') and not Name.str.contains('fdc')").shape:
    print('agual_log does not contain fdc or other mislabeld variables')
else:
    print('bad\n'*3+'agual_log DOES contain fdc or other mislabeld variables')

if rei.query("Group.str.contains('kenwood_log')").shape==rei.query("Group.str.contains('kenwood_log') and not Name.str.contains('fdc')").shape:
    print('kenwood_log does not contain fdc or other mislabeld variables')
else:
    print('\n'+'bad\n'*3+'kenwood_log DOES contain fdc or other mislabeld variables')
    
sfr = rei.query("Group.str.contains('aguacal_log')")
sfr.loc[:,'date'] = pd.to_datetime(sfr.Name.apply(lambda x:x.split("_date:")[-1]))
ax = sfr.query("Weight>0").plot(x = 'date', y = 'Modelled')
sfr.query("Weight>0").plot(x = 'date', y = 'Measured',ax = ax)
ax.set_title('Agua Caliente Minimum Flow per Water Year')
ax.set_ylabel("log(CFS)")
plt.savefig(os.path.join(out_folder, 'sfr_min_flow_agua_cal.png'), dpi =250, bbox_inches = 'tight')

sfr = rei.query("Group.str.contains('kenwood_log') and not Name.str.contains('fdc')")
sfr.loc[:,'date'] = pd.to_datetime(sfr.Name.apply(lambda x:x.split("_date:")[-1]))
ax = sfr.query("Weight>0").plot(x = 'date', y = 'Modelled')
sfr.query("Weight>0").plot(x = 'date', y = 'Measured',ax = ax)
ax.set_title('Kenwood Minimum Flow per Water Year')
ax.set_ylabel("log(CFS)")
plt.savefig(os.path.join(out_folder, 'sfr_min_flow_kenwood.png'), dpi =250, bbox_inches = 'tight')

In [ ]:
rei.Group.unique()

In [ ]:
flux = rei.query("Group=='flux' & not Name.str.contains('summ')")

flux.loc[:,'oname'] = flux.Name.apply(lambda x: x.split('oname:flux_otype:lst_usecol:')[1].split(':')[0])

flux.loc[:,'date'] = pd.to_datetime(flux.Name.apply(lambda x:x.split("_date:")[-1]))

In [ ]:
flux.set_index(['date','oname']).loc[:,['Modelled']].unstack().head()

In [ ]:
fl = flux.set_index(['date','oname']).loc[:,['Modelled']].unstack().filter(regex = "^(?!.*percent).*").filter(regex = "^(?!.*in_out_date).*")



fl = fl.droplevel(0,1)
columns_to_multiply = fl.filter(regex='_out_').columns

fl[columns_to_multiply] *= -1

stor =  fl.filter(regex = "storage")
stor = stor.mul(-1).sum(axis=1).cumsum()
fl = fl.filter(regex = "^(?!.*storage).*")

In [ ]:
fig, ax = conda_scripts.plot_help.stackedbar_wdates(fl.div(43560)*28, colormap='tab20')

ax2 = ax.twinx()



In [ ]:
rei.Group.unique()

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
q = rei.query("Group=='irr_total_volume'").copy()
q.loc[:,'date'] = q.Name.apply(lambda x: pd.to_datetime(x.split('_date:')[1][0:10]))
q.loc[:,'crop'] = q.Name.apply(lambda x: x.split('_crop:')[1])
q = q.set_index(['date','crop']).loc[:,["Modelled"]].unstack(1).droplevel(0,1)
fig, ax = conda_scripts.plot_help.stackedbar_wdates(q, colormap='tab20')
ax.set_title("Irrigation Volume Demand By Crop")
ax.set_ylabel('ft.')
plt.savefig(os.path.join(out_folder, 'irr_volumeby_crop.png'),bbox_inches = 'tight')

In [ ]:
from matplotlib import ticker

q = rei.query("Group=='irr_depth'").copy()
q.loc[:,'date'] = q.Name.apply(lambda x: pd.to_datetime(x.split('_date:')[1][0:10]))
q.loc[:,'crop'] = q.Name.apply(lambda x: x.split('_crop:')[1])
q = q.query( "crop =='vineyard'")
q = q.set_index(['date','crop']).loc[:,["Modelled"]].unstack(1).droplevel(0,1)
fig, ax = conda_scripts.plot_help.stackedbar_wdates(q, colormap='tab20')
ax.set_title('Irrigation Demand Depth');ax.set_ylabel('ft.')
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
plt.savefig(os.path.join(out_folder, 'irr_depthby_crop_vineyard.png'),bbox_inches = 'tight')

In [ ]:
os.makedirs(os.path.join(out_folder,'crop_info'), exist_ok=True)

In [ ]:
from matplotlib import ticker

q = rei.query("Group=='irr_depth' & Weight>0").copy()
q.loc[:,'date'] = q.Name.apply(lambda x: pd.to_datetime(x.split('_date:')[1][0:10]))
q.loc[:,'crop'] = q.Name.apply(lambda x: x.split('_crop:')[1])

q = q.set_index(['date','crop']).loc[:,["Modelled", 'Measured', 'Weight']].unstack(1)

os.makedirs(os.path.join(out_folder,'crop_info'), exist_ok=True)
for i, c in enumerate(q.loc[:,'Measured'].columns):
    plt.figure()
    ax = q.loc[:,'Modelled'].loc[:,c].plot(label = 'Modelled')
    ax.set_title(c)
    q.loc[:,'Measured'].loc[:,c].plot(ax = ax, c = 'blue', label = 'Measured')
    ax.set_title(f'Irrigation Demand Depth for {c}');
    ax.set_ylabel('ft.', c = 'blue'); ax.grid(True)
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.legend()
    ax2 = ax.twinx()
    ax2.set_ylabel('weight', color = 'g')
    q.loc[:,'Weight'].loc[:,c].plot(ax = ax2, c = 'g', ls = '--', label = 'Weight')
    ax2.legend(loc = 'lower right')
    plt.savefig(os.path.join(out_folder, 'crop_info', f'irr_depthby_{c}.png'),bbox_inches = 'tight')



# read actual outputs from bycroby farm

In [ ]:
# byfbyc = pd.read_csv(os.path.join(m_d, 'output', 'pump_by_farms.out' ),  sep = '\s+')
# byfbyc.loc[:,'DATE_START'] = pd.to_datetime(byfbyc.DATE_START).dt.year
# byfbyc = byfbyc.groupby(["DATE_START", 'WBS']).sum().loc[:,["Q-FIN"]].unstack().droplevel(0,1)


byfbyc = pd.read_csv(os.path.join(m_d, 'output', 'ByFarm_ByCrop.txt' ),  sep = '\s+')
byfbyc.loc[:,'DATE_START'] = pd.to_datetime(byfbyc.DATE_START).dt.year
byfbyc.head()

In [ ]:
byfbyc

In [ ]:
byfbyc


In [ ]:
byfbyc.query("CROP_NAME=='VINEYARD'").groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
       'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
       'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
       'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
       'PRECIPITATION',]].drop(columns = ['PRECIPITATION','TOT_SURF_RUNOFF',]).filter(regex = 'TRAN').loc['1970':,].plot(figsize = (15,15),subplots =True)

plt.savefig(os.path.join(out_folder, 'crop_info', f'transpiration_sum.png'),bbox_inches = 'tight')

In [ ]:
byfbyc.query("CROP_NAME=='VINEYARD'").groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
       'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
       'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
       'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
       'PRECIPITATION',]].drop(columns = ['PRECIPITATION','TOT_SURF_RUNOFF',]).filter(regex = 'TRAN').loc['1970':,].plot(figsize = (15,15),subplots =True)


In [ ]:
import flopy


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Create a figure
fig = plt.figure(figsize=(8.5, 11))

# Create gridspec layout with 1 row and 2 columns (one for left and one for right)
gs = gridspec.GridSpec(1, 1, width_ratios=[1])

# Create the left axes, which spans the entire height of the figure
ax3 = fig.add_subplot(gs[0])

sv_budget.sv_budget.sv_mod_map(simple = True,ax = ax3)

Map = flopy.plot.PlotMapView(model = ml, ax = ax3)
farms = os.path.join(workspace, 'fmp_input', 'farms','farm2016.dat')
farms = np.genfromtxt(farms, dtype = int)

subcat_farms = 81
farms = np.ma.array(farms, mask = farms<subcat_farms)
Map.plot_array(farms)
shp = conda_scripts.arich_functions.get_model_shp(ml.modelgrid)
shp = pd.merge(shp, conda_scripts.arich_functions.array2rc(farms).drop(columns = ['row', 'column']), on = ['i','j'])

conda_scripts.plot_help.label_points_outside_map(shp.set_geometry(shp.geometry.centroid).drop_duplicates('value').rename(columns = {'value':'label'}).query(f"label>{subcat_farms}"),fig, ax3, 'label' )

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def get_data(subcats = True):

    if subcats:
        data = byfbyc.query(f"WBS>={subcat_farms}")
    else:
        data = byfbyc.query(f"WBS<{subcat_farms}")
    # Example data (replace this with your pd.Series)
    
    data = data.groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
           'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
           'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
           'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
           'PRECIPITATION',]].loc['1970':,].mul(28/43560).mean()
    
    data.loc['IRR'] = data.loc['EVAP_IRR'] + data.loc['TRAN_IRR']
    data.loc['GW'] = data.loc['EVAP_GW'] + data.loc['TRAN_GW']
    return data

def plot_sarn(data, label, subcat):
    # Define the flows
    flows = [
        ("PRECIPITATION", "TOT_SURF_RUNOFF"),
        ("PRECIPITATION", "TOT_DEEP_PERC"),
        ("PRECIPITATION", "TRAN_PRECIP"),
        ("PRECIPITATION", "EVAP_PRECIP"),
        ("TRAN_PRECIP", "TRAN_SURF"),
        # ("TRAN_SURF", "TRAN"),
        ("TRAN_SURF", "SOIL_STRESS_LOSS"),
    
        ("GW", "EVAP_GW"),
        ("GW", "TRAN_GW"),
        ("IRR", "EVAP_IRR"),
        ("IRR", "TRAN_IRR")
    ]
    
    # Create unique lists of labels
    labels = sorted(data.index.tolist())
    
    # Create mappings from label to index
    label_indices = {label: i for i, label in enumerate(labels)}
    
    # Define the source and target indices and values
    sources = [label_indices[source] for source, target in flows]
    targets = [label_indices[target] for source, target in flows]
    values = [data[target] for source, target in flows]
    
    # Create the Sankey plot
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=50,
            thickness=30,
            line=dict(color="black", width=0.5),
            label=labels
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values
        )
    ))
    
    fig.update_layout(title_text=label, font_size=10)

    if subcat:
        sublabel = 'subcatchment'
    else:
        sublabel = 'non_subcatchment'
    
    fig.write_html(os.path.join(out_folder,f'SANKEY_bud_{sublabel}.html'))
    fig.show()

data = get_data(True)
data.to_csv(os.path.join(out_folder,f'SANKEY_bud_subcat.csv'))
plot_sarn(data, label = 'Hydrologic Flow Sankey Diagram for Subcatchments', subcat = True)
data = get_data(False)
data.to_csv(os.path.join(out_folder,f'SANKEY_bud_non_subcat.csv'))
plot_sarn(data, label = 'Hydrologic Flow Sankey Diagram for Non Subcatchments', subcat = False)

In [ ]:
byfbyc.query(f"WBS>={subcat_farms}").groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
       'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
       'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
       'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
       'PRECIPITATION',]].loc['1970':,].mul(28/43560).mean()

In [ ]:
byfbyc.query(f"WBS>={subcat_farms}").groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
       'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
       'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
       'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
       'PRECIPITATION',]].loc['1970':,].mul(28/43560).plot(figsize = (15,15),subplots =True)

plt.figure()
byfbyc.query(f"WBS>={subcat_farms}").groupby('DATE_START').sum().loc[:,[ 'TOT_DEEP_PERC', 'TOT_SURF_RUNOFF',
       'ADDED_DMD_DPERC', 'ADDED_DMD_RUNOFF', 'TRAN_POT', 'ANOXIA_LOSS',
       'SOIL_STRESS_LOSS', 'TRAN', 'TRAN_SURF_INI', 'TRAN_SURF', 'TRAN_IRR',
       'TRAN_PRECIP', 'TRAN_GW', 'EVAP_IRR', 'EVAP_PRECIP', 'EVAP_GW',
       'PRECIPITATION',]].loc['1970':,].mul(28/43560).mean().plot.barh(figsize = (15,15),subplots =False)

In [ ]:
info = pd.read_csv(os.path.join('..','waterlevel', 'GWLE','gwle_station_info.csv'),index_col=0)
info.loc[:,'station'] = info.loc[:,'Station Name'].str.lower()
info = gpd.GeoDataFrame(info, geometry = gpd.points_from_xy(info.Easting, info.Northing, crs = 2226),).to_crs(2226)


other_info = pd.read_csv(os.path.join('..','waterlevel', 'stats.csv'),index_col=0)
other_info.loc[:,'station'] = other_info.loc[:,'station_name'].str.lower()
other_info = other_info.loc[:,['station', 'station_latitude','station_longitude']]
other_info = gpd.GeoDataFrame(other_info, geometry = gpd.points_from_xy(other_info.station_longitude, other_info.station_latitude, crs = 4326),).to_crs(2226)
other_info.loc[:,'Easting'] = other_info.geometry.x
other_info.loc[:,'Northing'] = other_info.geometry.y
other_info = other_info.loc[~other_info.loc[:,'station'].isin(info.loc[:,'station'])]
info = pd.concat([info, other_info])

info.plot()

In [ ]:
zones = helpers.get_zones(ml)

In [ ]:
from shapely import Point
from matplotlib.gridspec import GridSpec
def create_figure(station):
    
    fig = plt.figure(figsize = (10,6))
    
    gs1 = GridSpec(2, 2,  left=0.05, right=0.48, wspace=0.05,width_ratios=[7, 2],)
    ax = fig.add_subplot(gs1[:, 0])
    ax.tick_params(labelbottom=False, labelleft=True)

    
    ax3 = fig.add_subplot(gs1[1:, -1])
    ax3.tick_params(labelbottom=False, labelleft=False)
    sv_budget.sv_budget.sv_mod_map(simple = True,ax = ax3)
    annotations = [child for child in ax3.get_children() if isinstance(child, mpl.text.Text)]
    annotations[0].remove()
    ax3.legend().remove()

    
    xy = info.query(f"station=='{station}'")

    if xy.shape[0]==0:
        pass
    else:
        pt = gpd.GeoSeries(Point(xy.Easting, xy.Northing),crs = 2226)
        pt.plot(ax = ax3,  markersize = 40,marker = '*', color = 'r') 
        


    return fig, ax, ax3

def fmt_axis(ax,station, zone):
    '''plot model layers'''
    ax.set_title(station+f' ' + zone,loc="left")
    ax.grid(True)

    arr = np.vstack([np.expand_dims(ml.dis.top.array, 0), ml.dis.botm.array])
    ylim = ax.get_ylim()
    if info.query(f"station=='{station}'").shape[0]>0:
        i = info.query(f"station=='{station}'").i.unique()[0]
        j = info.query(f"station=='{station}'").j.unique()[0]

        #print model layers
        if isinstance(i, np.int64):
            verts = arr[:,i,j]
            verts = list(filter(lambda x: x> ylim[0], verts))
            for l in verts:
                ax.axhline(l, ls = '--', c = 'grey', alpha = .6, zorder = 0)
                
            conda_scripts.plot_help.label_outside_multiple(ax, y = verts, labels = [f'lay{lay}' for lay in np.arange(len(verts))])

    ph.yrange_(ax)

station = 'son0001'
fig, ax, axmap = create_figure(station)
fmt_axis(ax, station, 'asdf')

In [ ]:
def get_phi(g,  ax ):
    # g = g.query(f"Group.str.contains('{name}') & Weight>0")

    phisum = g.Weight* g.Residual.apply(lambda x: x**2)
    phisum = phisum.sum()
    ax.text(0.95, 0.05, f'{phisum:,.1f}', 
        transform=ax.transAxes, 
        fontsize=12, 
        verticalalignment='bottom', 
        horizontalalignment='right',
        bbox=dict(facecolor='white', alpha=0.5))
    
    return 

In [ ]:
hds =  rei.query("Group.str.contains('hds') or Group.str.contains('gwle')").copy()
hds.loc[:,'date'] = pd.to_datetime(hds.Name.apply(lambda x:x.split("_date:")[-1]))

hds.loc[:,'station'] = hds.Name.apply(lambda x: x.split(':')[4].split(':')[0].replace('_date',''))

for station, group in hds.groupby('station'):

    fig, ax, axmap = create_figure(station)

    if group.query("Group.str.contains('gwle_abs') & Weight>0 ").shape[0]>0:
        ax = group.query("Group.str.contains('gwle_abs') & Weight>0 ").plot(x = 'date', y = 'Modelled', label = 'gwle_abs - modeled', ax = ax)
        group.query("Group.str.contains('gwle_abs') & Weight>0 ").plot(x = 'date', y = 'Measured', ax = ax, label = "gwle_abs - 'Observed'")

        
    if group.query("Group.str.contains('hds') & Weight>0 ").shape[0]>0:
        if ax is None:
            fig, ax = plt.subplots()
        group.query("Group.str.contains('hds') & Weight>0 ").plot(x = 'date', y = 'Modelled', ax = ax,  marker = 'o', label = 'hds - modeled')
        group.query("Group.str.contains('hds') & Weight>0 ").plot(x = 'date', y = 'Measured',  ax = ax, marker = 'o',label = 'hds - Observed')


    if group.query("Group.str.contains('gwle_roll18month') & Weight>0 ").shape[0]>0:
        if ax is None:
            fig, ax = plt.subplots()
        group.query("Group.str.contains('gwle_roll18month') & Weight>0 ").plot(x = 'date', y = 'Modelled', ax = ax,  mfc = 'w', lw = .2,
                                                            marker = 'o', label = 'gwle_roll18month - modeled')
        group.query("Group.str.contains('gwle_roll18month') & Weight>0 ").plot(x = 'date', y = 'Measured',  ax = ax,mfc = 'w', lw = .2,
                                                            marker = 'o',label = 'gwle_roll18month - Observed')

        
    if group.query("Group.str.contains('gwle_roll_') & Weight>0 ").shape[0]>0:
        if ax is None:
            fig, ax = plt.subplots()
        group.query("Group.str.contains('gwle_roll_') & Weight>0 ").plot(x = 'date', y = 'Modelled', ax = ax,  mfc = 'orange',
                                                            marker = 'o', label = 'gwle roll - modeled')
        group.query("Group.str.contains('gwle_roll_') & Weight>0 ").plot(x = 'date', y = 'Measured',  ax = ax,mfc = 'orange',
                                                            marker = 'o',label = 'gwle roll - Observed')




    if group.query("Group.str.contains('gwle_majordrawdown & Weight>0 ') & Weight>0").shape[0]>0:
        if ax is None:
            fig, ax = plt.subplots()
        group.query("Group.str.contains('gwle_majordrawdown') & Weight>0").plot(x = 'date', y = 'Modelled', ax = ax,  mfc = 'k',
                                                            marker = 'o', label = 'gwle_majordrawdown - modeled')
        group.query("Group.str.contains('gwle_majordrawdown') & Weight>0").plot(x = 'date', y = 'Measured',  ax = ax,mfc = 'k',
                                                            marker = 'o',label = 'gwle_majordrawdown - Observed')
        
    get_phi(group,  ax )
    
    zone = group.Group.unique()[0].split("_")[-1]
    print(station+' '+ zone)
    ax.set_title(station+' '+ group.Group.unique()[0].split("_")[-1])

    fmt_axis(ax, station, 'Heads')

    print(os.path.join(out_folder, 'hydros','main', zone, station+'.png'))
    os.makedirs(os.path.join(out_folder, 'hydros','main', zone), exist_ok = True)
    plt.savefig(os.path.join(out_folder, 'hydros','main', zone, station+'.png'))
    os.makedirs(os.path.join(out_folder, 'hydros','main', 'all'), exist_ok = True)
    plt.savefig(os.path.join(out_folder, 'hydros','main', 'all', station+'.png'))
    plt.close()
    

# make map of starting head residuals

In [ ]:
import mapclassify

In [ ]:
hds =  rei.query("Group.str.contains('gwle_abs')").copy()
hds.loc[:,'date'] = pd.to_datetime(hds.Name.apply(lambda x:x.split("_date:")[-1]))

hds.loc[:,'station'] = hds.Name.apply(lambda x: x.split(':')[4].split(':')[0].replace('_date',''))


hds = pd.merge(hds, info, on = 'station', how = 'left')

hds = hds.loc[(hds.date.dt.year==1975) &( hds.date.dt.month==2) ]

head_diff = gpd.GeoDataFrame(hds, geometry = gpd.points_from_xy(hds.Easting, hds.Northing), crs = 2226)

q10 = mapclassify.Quantiles(head_diff.loc[:,'Residual'].abs(), k=10)
head_diff = head_diff.assign(cl=q10.yb)

In [ ]:
head_diff

In [ ]:
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(figsize = (8.5,11))

gs1 = GridSpec(1, 1,  left=0.05, right=0.48, wspace=0.05)



ax3 = fig.add_subplot(gs1[:, :], projection = ccrs.epsg(2226))
ax3.tick_params(labelbottom=False, labelleft=False)
sv_budget.sv_budget.sv_mod_map(simple = True,ax = ax3)

helpers.plot_zones(ml,fig = fig, ax = ax3, zones = zones, label = True)

In [ ]:
from matplotlib.colors import TwoSlopeNorm
# normalize color
vmin, vmax, vcenter = head_diff.loc[:,'Residual'].min(), head_diff.loc[:,'Residual'].max(), 0
norm = TwoSlopeNorm(vmin=-100, vcenter=vcenter, vmax=100)
# create a normalized colorbar
cmap = 'RdBu'
cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

## plot all obbs from all layers
fig = plt.figure(dpi = 250)
ax = sv_budget.sv_budget.sv_mod_map(simple = True,figsize = (8.5,11))
ax.set_title('Deep Wells')
helpers.plot_zones(ml,fig = fig, ax = ax, zones = zones, label = False)
head_diff.loc[:,'ms'] = head_diff.loc[:,'cl']*5+5
head_diff.query("Depth=='Deep'").plot(ax=ax, column = 'Residual', markersize ='ms', cmap=cmap, norm=norm, ec = 'k' )
fig.colorbar(cbar, ax=ax,shrink = .5, extend = 'both')

_ = ph.label_points(ax, head_diff.query("Depth=='Deep'"), 'Residual',already_str=False, fmt = '+.0f', iter_limit= 1000,
                                        kwds_dict={'x':head_diff.query("Depth=='Deep'").geometry.x.values,'y':head_diff.query("Depth=='Deep'").geometry.y.values,
                                'avoid_self':True,
                                'expand':(1.5,1.5)})

plt.savefig(os.path.join(out_folder,'mean_map_resid_deep.png'))

In [ ]:



## plot all obbs from all layers
fig = plt.figure(dpi = 250)
ax = sv_budget.sv_budget.sv_mod_map(simple = True,figsize = (8.5,11))
ax.set_title('Shallow Wells')
helpers.plot_zones(ml,fig = fig, ax = ax, zones = zones, label = False)
head_diff.loc[:,'ms'] = head_diff.loc[:,'cl']*5+5
head_diff.query("Depth=='Shallow'").plot(ax=ax, column = 'Residual', markersize ='ms', cmap=cmap, norm=norm, ec = 'k' )
_ = ph.label_points(ax, head_diff.query("Depth=='Shallow'"), 'Residual',already_str=False, fmt = '+.0f', iter_limit= 10000,
                                        kwds_dict={'x':head_diff.query("Depth=='Shallow'").geometry.x.values,
                                                   'y':head_diff.query("Depth=='Shallow'").geometry.y.values,
                                'avoid_self':False,
                                'expand':(2.9,1.9)})

fig.colorbar(cbar, ax=ax,shrink = .5, extend = 'both')

# _ = ph.label_points(ax, head_diff.query("Depth=='Shallow'"), 'Residual',already_str=False, fmt = '+.0f', iter_limit= 1000)


plt.savefig(os.path.join(out_folder,'mean_map_resid_shallow.png'))


In [ ]:
rei.query("Group=='irr_depth'")

In [ ]:
import sys
from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

proj_root = Path.cwd().parent.parent

import flopy

print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mpl.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"flopy version: {flopy.__version__}")



In [ ]:
# Create a ZoneBudget object and get the budget record array
zb = flopy.utils.ZoneBudget(cbc_f, zon, kstpkper=(0, 1096))
zb.get_budget()

In [ ]:
zones = np.genfromtxt(zone_file, dtype=int)

plt.imshow(zones)

In [ ]:
bas.geometry.

In [ ]:
fb_depth.mean().to_dict()

In [ ]:


fb_depth = forward_run.crop_irr_depth(workspace)
fb_total = forward_run.total_irr_demand(workspace)

conda_scripts.plot_help.stackedbar_wdates(fb_depth)
conda_scripts.plot_help.stackedbar_wdates(fb_total)


In [ ]:

len('NATIVE_VEGETATION_RI')

In [ ]:
plt.imshow(zone[1])

In [ ]:

import time

In [ ]:
zb.loc[:,zb.filter(regex = 'TO_').columns]= zb.loc[:,zb.filter(regex = 'TO_').columns]*-1

In [ ]:
zb = zb.drop(columns = {'FROM_STORAGE','TO_STORAGE','TOTAL_IN','TOTAL_OUT'}, errors = 'ignore' )

In [ ]:

conda_scripts.plot_help.stackedbar_wdates(zb.filter(regex = "FARM"), colormap = 'tab20')

In [ ]:
conda_scripts.plot_help.stackedbar_wdates(zb, colormap = 'tab20')

In [ ]:
from flopy.utils import ZoneBudget

z = np.genfromtxt(os.path.join(ml.model_ws, 'model_arrays', 'zonation_3.csv'), delimiter = ' ')


nlay, nrow, ncol = zon.shape

fig = plt.figure(figsize=(10, 4))

for lay in range(nlay):
    ax = fig.add_subplot(1, nlay, lay + 1)
    im = ax.pcolormesh(zon[lay, ::-1, :])
    cbar = plt.colorbar(im)
    plt.gca().set_aspect("equal")

plt.show()



In [ ]:
def zone_bud(ml):

    

    
    

In [ ]:
pst.parameter_data.query("parnme.str.contains('drn')").tail(20)